Auteur: Michaël Leroy

 Graph depuis gdf Nodes et edges
        - calcul de stats
        - classif des communes 

        .....

In [1]:
class CFG:
    data_path = './osm_datas/'
    data_path_mappers = './datas'
    paths_path = './tourism_paths'
    area_name = 'france'

    
import os
os.environ['USE_PYGEOS'] = '0'

import joblib
# joblib.dump(
#     CFG, 
#     os.path.join(
#         CFG.paths_path,
#         'CFG_for_tourism_paths.pkl'
#     ) 
# )   


# %load_ext autoreload
# %autoreload


# %aimport OSM_helper_functions
# %aimport OCM_helper_functions

import numpy as np
import geopandas as gpd
import pandas as pd
import osmnx as ox
import networkx as nx
from tqdm.notebook import tqdm
# from OSM_helper_functions import *
# from OCM_helper_functions import *


ox.settings.log_console=True
ox.settings.use_cache=True
''' To retrieve http data from OSMnx and cache them locally
    before building graph. Raise and exception if requests
    are all done. intercept then set to False and retry.'''
# ox.settings.cache_only_mode=True 



C:\Users\demo\AppData\Roaming\Python\Python39\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.25.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


' To retrieve http data from OSMnx and cache them locally\n    before building graph. Raise and exception if requests\n    are all done. intercept then set to False and retry.'

In [2]:
Nodes = joblib.load(os.path.join(CFG.data_path, f'[{CFG.area_name}]Gnodes_communes.pkl'))

Edges = joblib.load(os.path.join(CFG.data_path, f'[{CFG.area_name}]Gedges_communes_C22.pkl'))

In [3]:
# TODO: Make a choice for int or str for nodes ids
# Nodes.index = Nodes.index.astype(int)

In [4]:
Nodes.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 34466 entries, 2531584 to 2741108
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   y         34466 non-null  float64 
 1   x         34466 non-null  float64 
 2   codgeo    34466 non-null  object  
 3   area      34466 non-null  object  
 4   sleeping  15412 non-null  float64 
 5   visit     17923 non-null  float64 
 6   fooding   6616 non-null   float64 
 7   working   6950 non-null   float64 
 8   vehicule  4166 non-null   float64 
 9   geometry  34466 non-null  geometry
dtypes: float64(7), geometry(1), object(2)
memory usage: 3.9+ MB


In [5]:
Edges = Edges.drop(['src_trgt'], axis=1)
Edges = Edges.reset_index()

Edges['u'] = Edges['u'].astype(str)
Edges['v'] = Edges['v'].astype(str)
Edges['key'] = Edges['key'].astype(str)

Edges = Edges.set_index(['u','v','key'])
PATHS = Edges.pop('path')
Edges.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
MultiIndex: 205450 entries, ('2531584', '2176035', '0') to ('2741108', '2740963', '0')
Data columns (total 4 columns):
 #   Column       Non-Null Count   Dtype   
---  ------       --------------   -----   
 0   geometry     205450 non-null  geometry
 1   length       205450 non-null  float64 
 2   travel_time  205450 non-null  float64 
 3   path_line    205450 non-null  object  
dtypes: float64(2), geometry(1), object(1)
memory usage: 10.6+ MB


In [6]:
nodes2codgeo = dict(zip(Nodes.index, Nodes['codgeo']))

In [7]:
display(Nodes.head(2),Edges.head(2))

,y,x,codgeo,area,sleeping,visit,fooding,working,vehicule,geometry
osmid,,,,,,,,,,
2531584,43.461438,-0.819766,64312,POLYGON ((-0.832953105776137 43.44755970632465...,2.0,NaN,NaN,NaN,NaN,POINT (-0.81977 43.46144)
279376,47.730421,0.777490,41255,POLYGON ((0.7426134063245691 47.72901158483156...,NaN,1.0,NaN,NaN,NaN,POINT (0.77749 47.73042)


geometry  \
u       v       key                                                      
2531584 2176035 0    LINESTRING (-0.81977 43.46144, -0.84879 43.43624)   
        167704  0    LINESTRING (-0.81977 43.46144, -0.77384 43.48750)   

                       length  travel_time  \
u       v       key                          
2531584 2176035 0    6669.304        466.3   
        167704  0    6019.824        403.2   

                                                             path_line  
u       v       key                                                     
2531584 2176035 0    LINESTRING (-0.820294 43.4613392, -0.819854 43...  
        167704  0    LINESTRING (-0.820294 43.4613392, -0.819854 43...

In [8]:
# Mappers to link osmids to commune names


# Dict to map osmid  -> communes names
Osmid2Name = joblib.load(
    os.path.join(CFG.data_path_mappers,f'[{CFG.area_name}]mapper_osmid_to_libgeo.pkl')
)

# Dict to map communes names  -> osmid 
Name2Osmid = joblib.load(
    os.path.join(CFG.data_path_mappers,f'[{CFG.area_name}]mapper_libgeo_to_osmid.pkl')
)

------------------------------------------

In [11]:
not_connected_nodes = set (Nodes.index) - set(Edges.reset_index().u) - set(Edges.reset_index().v)
not_connected_nodes

{'107972',
 '113454',
 '156066',
 '192873',
 '215400',
 '215401',
 '278723',
 '399669',
 '417923',
 '659733',
 '900558',
 '960821'}

In [12]:
not_connected_communes = [Osmid2Name[node] for node in not_connected_nodes]
dict(zip(not_connected_nodes,not_connected_communes))

{'960821': 'ÎLE-AUX-MOINES',
 '156066': "L'ÎLE-D'YEU",
 '417923': 'ÎLE-DE-BATZ',
 '399669': 'ÎLE-DE-BRÉHAT',
 '107972': 'GROIX',
 '192873': "ÎLE-D'ARZ",
 '659733': 'OUESSANT',
 '215401': 'HŒDIC',
 '900558': 'ÎLE-MOLÈNE',
 '113454': "ÎLE-D'AIX",
 '215400': "ÎLE-D'HOUAT",
 '278723': 'ÎLE-DE-SEIN'}

-------------------------------------------